In [2]:
# Install missing libraries

# !pip install trafilatura
# !pip install --user -U nltk

  Using cached trafilatura-1.4.0-py3-none-any.whl (1.0 MB)
  Using cached courlan-0.8.3-py3-none-any.whl (34 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.1/122.1 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Using cached htmldate-1.3.2-py3-none-any.whl (39 kB)
  Using cached jusText-3.0.0-py2.py3-none-any.whl (837 kB)
  Using cached tld-0.12.6-py38-none-any.whl (412 kB)
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 2.0.12
    Uninstalling charset-normalizer-2.0.12:
      Successfully uninstalled charset-normalizer-2.0.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.27.1 requires charset-normalizer~=2.0.0; python_version >= "3", but you have charset-normalizer 3.0.0 which is incompatible.
apify-client 0.5.0 requires requests~=2.26.0, but you have requests 2.27.1 which is incompatible.
aiohttp 3.8.1 requ

In [11]:
# Import libraries
from trafilatura import fetch_url, extract
import json
from collections import Counter
import re
import string
import nltk
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import spacy
import requests

In [10]:
websiteUrl = 'https://30dumi.com/'
ploads = {'url':websiteUrl}
r = requests.post('http://localhost:8080/api/html', json=ploads)

pattern = r'(?<=href=")(?=https:\/\/eadsrv.com).+?(?=")'
print(r.text)
text = r.text
ad = re.search(pattern, text)
print(ad)
adOrigin = ad.group()

<!DOCTYPE html><html dir="ltr" lang="bg-BG" prefix="og: https://ogp.me/ns#" class="js svg background-fixed"><head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="http://gmpg.org/xfn/11">
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<title>Оптимизация на сайтове за търсачки, SEO оптимизация - СЕО агенция 30 думи - София</title>
<!-- All in One SEO 4.2.5.1 - aioseo.com -->
<meta name="robots" content="max-image-preview:large">
<link rel="canonical" href="https://30dumi.com">
<meta name="generator" content="All in One SEO (AIOSEO) 4.2.5.1 ">
<meta property="og:locale" content="bg_BG">
<meta property="og:site_name" content="Оптимизация на сайтове за търсачки, SEO оптимизация - СЕО агенция 30 думи - София | Оптимизираните сайтове са посещавани от стотици хора без реклама! Не евтино, а безплатно!">
<meta property="og:type" content="activity">
<meta

AttributeError: 'NoneType' object has no attribute 'group'

In [9]:
# Function to get the full text of any ad website
def get_text_ad_page(link):
    downloaded = fetch_url(link)
    result = extract(downloaded, output_format="json", include_links=False)
    json_result = json.loads(result)
    # print(json_result['raw_text'])
    return json_result['raw_text']

In [5]:
# translate text through DeepL
def translate_deepl(text):
    response = requests.post(
    'https://api-free.deepl.com/v2/translate',
    params={ 
    # remove auth key before commiting to repo
    "auth_key": "AUTH_KEY", 
    "target_lang": "EN", 
    "text": text, 
    }, 
    )
    result = response.json()["translations"][0]["text"]
    return result

In [6]:
# Import all dictionaries
with open('dictionaries/Други.txt') as f:
    others = [line.rstrip('\n') for line in f]
with open('dictionaries/Козметика.txt') as f:
    cosmetics = [line.rstrip('\n') for line in f]
with open('dictionaries/Косопад.txt') as f:
    hairloss = [line.rstrip('\n') for line in f]
with open('dictionaries/Общи.txt') as f:
    general = [line.rstrip('\n') for line in f]
with open('dictionaries/Отслабване.txt') as f:
    weight_loss = [line.rstrip('\n') for line in f]


In [24]:
# test
raw_text = get_text_ad_page("https://www.dnevnik.bg/v_pamet/2022/11/13/4414932_pochina_djaz_muzikantut_emanuil_manolov_-badema/?ref=home_NaiNovoto")
translated_text = translate_deepl(raw_text)

JSONDecodeError: [Errno Expecting value] <html><body><h1>400 Bad request</h1>
Your browser sent an invalid request.
</body></html>
: 0

In [21]:
translated_text

'One of the legends of Bulgarian jazz, saxophonist Emanuil Manolov - Badema, passed away yesterday - November 12, 2022 at the age of 87, the family announced. His name is associated with the orchestras of Eddie Kazasyan and Lea Ivanova, Vasko Alipiev, Horst Kruger - sextet, Klaus Lenz. He has played in Lili Ivanova\'s group "Poppies" and as a soloist in the Bulgarian National Radio\'s Big Band under the direction of Vili Kazasyan. He was also the leader of the Dixie Swing Orchestra. The funeral will take place on Wednesday, 16.11.2022 in St. Sofia Church at 12.00. We bow to his memory.'

In [23]:
# Text pre-processing
nlp = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()
nltk.download('stopwords')
stopwords_english = stopwords.words('english')
doc = nlp(translated_text)

text_clean = []

for sentence in doc.sents:
    sentence = re.sub(r'https?://[^\s\n\r]+', '', str(sentence))
    # remove hashtags
    # only removing the hash # sign from the word
    sentence = re.sub(r'#', '', str(sentence))
    # tokenize comments
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    sentence_tokens = tokenizer.tokenize(sentence)

    sentence_clean = []
    for word in sentence_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            text_clean.append(stem_word)

print(text_clean)


OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Count words 
gram_df = pd.Series(nltk.ngrams(filtered_sentence, ngrams)).value_counts()

In [ ]:
# Map each word to a category and count number of categories